# Feature Selection

***** FILL OUT LATER *****

*** DELETE NOTES AFTER COMPLETION ***

NOTES:
- Batters will predict on 20 features
- Pitchers will predict on 25 features

CURRENT ISSUES: 
- Running into error when trying to pad dataset with zeros
- ValueError: could not convert string to float: 'Alan\xa0Foster'


In [1]:
#####
# - IMPORTS
#####

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

In [2]:
#####
# - LOADING DATASETS & SETTING IT SO THAT ALL COLUMNS SHOW WHEN VIEWING THE DATASETS
#####

batting = pd.read_csv('clean_batting.csv',index_col=0)
pitching = pd.read_csv('clean_pitching.csv',index_col=0)

pd.set_option('display.max_columns', None)

In [3]:
batting

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO
0,A.J. Burnett,24,FLA,25,59,50,0,4,1,0,0,0,0,0,1,27,0.080,0.115,0.100,0.215,-42,5,0,1,7,0,0,2001,0.105,3.0,1.0,5.0,28.0
1,A.J. Burnett,25,FLA,29,69,57,1,6,2,0,1,3,0,0,5,28,0.105,0.177,0.193,0.370,-1,11,1,0,7,0,0,2002,0.138,1.0,0.0,0.0,11.0
2,A.J. Burnett,27,FLA,18,37,29,4,4,0,0,0,1,0,0,0,11,0.138,0.138,0.138,0.276,-26,4,1,0,8,0,0,2004,0.147,2.0,1.0,1.0,34.0
3,A.J. Burnett,28,FLA,31,79,68,3,10,2,2,1,2,0,0,1,34,0.147,0.171,0.279,0.451,19,19,1,1,9,0,0,2005,0.063,2.0,0.0,3.0,36.0
4,A.J. Burnett,35,PIT,29,71,63,1,4,0,0,0,2,0,0,3,36,0.063,0.106,0.063,0.170,-51,4,1,0,5,0,0,2012,0.068,2.0,0.0,2.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31688,Ángel Pagán#,29,NYM,123,532,478,68,125,24,4,7,56,32,7,44,62,0.262,0.322,0.372,0.694,94,178,4,1,4,5,4,2011,0.288,56.0,8.0,48.0,97.0
31689,Ángel Pagán#,30,SFG,154,659,605,95,174,38,15,8,56,29,7,48,97,0.288,0.338,0.440,0.778,120,266,6,0,2,4,5,2012,0.282,30.0,5.0,23.0,36.0
31690,Ángel Pagán#,31,SFG,71,305,280,44,79,16,3,5,30,9,4,23,36,0.282,0.334,0.414,0.749,113,116,1,0,0,2,0,2013,0.300,27.0,3.0,25.0,53.0
31691,Ángel Pagán#,32,SFG,96,413,383,56,115,21,2,3,27,16,6,25,53,0.300,0.342,0.389,0.731,110,149,5,1,1,3,1,2014,0.262,37.0,3.0,32.0,93.0


In [4]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV
0,A.J. Achter,25,MIN,1,0,1.000,3.27,7,0,1,0,0,0,11.0,14,7,4,2,3,0,5,0,0,0,49,123,5.40,1.545,11.5,1.6,2.5,4.1,1.67,2014,6.75,14.0,1.350,6.0,0.0,0.0
1,A.J. Achter,26,MIN,0,1,0.000,6.75,11,0,4,0,0,0,13.1,12,10,10,4,6,1,14,0,0,0,58,62,6.28,1.350,8.1,2.7,4.1,9.5,2.33,2015,3.11,14.0,1.460,12.0,1.0,0.0
3,A.J. Burnett,22,FLA,4,2,0.667,3.48,7,7,0,0,0,0,41.1,37,23,16,3,25,2,33,0,0,0,182,124,4.30,1.500,8.1,0.7,5.4,7.2,1.32,1999,4.79,57.0,1.500,44.0,3.0,0.0
4,A.J. Burnett,23,FLA,3,7,0.300,4.79,13,13,0,0,0,0,82.2,80,46,44,8,44,3,57,2,0,2,364,92,4.68,1.500,8.7,0.9,4.8,6.2,1.30,2000,4.05,128.0,1.315,83.0,11.0,0.0
5,A.J. Burnett,24,FLA,11,12,0.478,4.05,27,27,0,2,1,0,173.1,145,82,78,20,83,3,128,7,1,7,733,105,4.63,1.315,7.5,1.0,4.3,6.6,1.54,2001,3.30,203.0,1.189,90.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,31,PIT,8,8,0.500,3.06,23,22,0,4,3,0,141.0,138,56,48,8,19,3,56,2,0,0,566,114,3.17,1.113,8.8,0.5,1.2,3.6,2.95,1992,4.55,32.0,1.434,22.0,3.0,0.0
23610,Zane Smith*,32,PIT,3,7,0.300,4.55,14,14,0,1,0,0,83.0,97,43,42,5,22,3,32,0,0,2,353,88,3.80,1.434,10.5,0.5,2.4,3.5,1.45,1993,3.27,57.0,1.248,34.0,10.0,0.0
23611,Zane Smith*,33,PIT,10,8,0.556,3.27,25,24,0,2,1,0,157.0,162,67,57,18,34,7,57,0,0,2,645,132,4.55,1.248,9.3,1.0,1.9,3.3,1.68,1994,5.61,47.0,1.509,23.0,8.0,0.0
23612,Zane Smith*,34,BOS,8,8,0.500,5.61,24,21,0,0,0,0,110.2,144,78,69,7,23,1,47,1,1,0,484,87,3.73,1.509,11.7,0.6,1.9,3.8,2.04,1995,5.08,47.0,1.500,21.0,4.0,0.0


In [5]:
#####
# - SETTING UP THE FEATURE SELECTORS
#####

rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

bat_sfs_ba = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_rbi = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_hr = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_bb = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_so = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 

pitch_sfs_era = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_so = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_whip = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_bb = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_w = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_sv = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 

In [6]:
#####
# - REMOVING STRING COLUMNS ALONG WITH COLUMNS TO COMPARE PREDICTIONS TO
#####

remove_bat_cols = ['Name', 'Tm', 'Year', 'Nxt_BA', 'Nxt_RBI', 'Nxt_HR', 'Nxt_BB', 'Nxt_SO']
remove_pitch_cols = ['Name', 'Tm', 'Year', 'Nxt_ERA', 'Nxt_SO', 'Nxt_WHIP', 'Nxt_BB', 'Nxt_W', 'Nxt_SV']

bat_cols = batting.columns[~batting.columns.isin(remove_bat_cols)]
pitch_cols = pitching.columns[~pitching.columns.isin(remove_pitch_cols)]

In [7]:
#####
# - SCALING THE DATA
#####

scaler = MinMaxScaler()
batting.loc[:, bat_cols] = scaler.fit_transform(batting[bat_cols])
pitching.loc[:, pitch_cols] = scaler.fit_transform(pitching[pitch_cols])

In [ ]:
batting

In [ ]:
pitching

In [8]:
#####
# - FITTING EACH OF THE PREDICTORS
#####

bat_sfs_ba.fit(batting[bat_cols], batting['Nxt_BA'])
bat_sfs_rbi.fit(batting[bat_cols], batting['Nxt_RBI'])
bat_sfs_hr.fit(batting[bat_cols], batting['Nxt_HR'])
bat_sfs_bb.fit(batting[bat_cols], batting['Nxt_BB'])
bat_sfs_so.fit(batting[bat_cols], batting['Nxt_SO'])

pitch_sfs_era.fit(pitching[pitch_cols], pitching['Nxt_ERA'])
pitch_sfs_so.fit(pitching[pitch_cols], pitching['Nxt_SO'])
pitch_sfs_whip.fit(pitching[pitch_cols], pitching['Nxt_WHIP'])
pitch_sfs_bb.fit(pitching[pitch_cols], pitching['Nxt_BB'])
pitch_sfs_w.fit(pitching[pitch_cols], pitching['Nxt_W'])
pitch_sfs_sv.fit(pitching[pitch_cols], pitching['Nxt_SV'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=25,
                          n_jobs=-1)

In [9]:
#####
# - SETTING UP LIST OF PREDICTORS FOR EACH FEATURE
#####

bat_ba_preds = list(bat_cols[bat_sfs_ba.get_support()])
bat_rbi_preds = list(bat_cols[bat_sfs_rbi.get_support()])
bat_hr_preds = list(bat_cols[bat_sfs_hr.get_support()])
bat_bb_preds = list(bat_cols[bat_sfs_bb.get_support()])
bat_so_preds = list(bat_cols[bat_sfs_so.get_support()])

pitch_era_preds = list(pitch_cols[pitch_sfs_era.get_support()])
pitch_so_preds = list(pitch_cols[pitch_sfs_so.get_support()])
pitch_whip_preds = list(pitch_cols[pitch_sfs_whip.get_support()])
pitch_bb_preds = list(pitch_cols[pitch_sfs_bb.get_support()])
pitch_w_preds = list(pitch_cols[pitch_sfs_w.get_support()])
pitch_sv_preds = list(pitch_cols[pitch_sfs_sv.get_support()])

In [10]:
bat_ba_preds

['Age',
 'G',
 'PA',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'CS',
 'BB',
 'SO',
 'OBP',
 'OPS',
 'TB',
 'GDP',
 'SH',
 'SF',
 'IBB']

In [ ]:
bat_rbi_preds

In [ ]:
bat_hr_preds

In [ ]:
bat_bb_preds

In [ ]:
bat_so_preds

In [ ]:
pitch_era_preds

In [ ]:
pitch_so_preds

In [ ]:
pitch_whip_preds

In [ ]:
pitch_bb_preds

In [ ]:
pitch_w_preds

In [ ]:
pitch_sv_preds

# WORK IN PROGRESS

## Figuring out how to code the RNN model

### Will need to do this in a multivariable way. Will need to figure out


In [14]:
#####
# - IMPORTS
#####
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [15]:
#####
# - BUILDING MULTIVARIATE TIME SERIES LSTM MODEL
#####

def bat_ba_mrnn (data, model, predictors, start=2, step=1):
    bat_preds = []
    
    seasons = sorted(data["Year"].unique())
    
    for i in range(start, len(seasons), step):
        current_season = seasons[i]
        train = data[data['Year'] < current_season]
        test = data[data['Year'] == current_season]
        
        
        model = Sequential()
        
        #train = train.values.reshape(-1, 33, 20)

        zeros = np.zeros((2,20))
        zeros[:train.shape[0],:train.shape[1]] = train
        
        model.add(LSTM(units = 175, return_sequences = True, input_shape = (33,20)))
        model.add(Dropout(0.25))
        model.add(LSTM(units = 142, return_sequences = True, dropout=0.2,recurrent_dropout=0.15))
        model.add(LSTM(units = 125, return_sequences = True, dropout=0.2,recurrent_dropout=0.15))
        model.add(LSTM(units = 100, return_sequences = True, dropout=0.2,recurrent_dropout=0.15))
        model.add(LSTM(units = 75, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 75, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= False))
        model.add(Dense(units = 1))  

        
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
        model.fit(train[predictors], train['Nxt_BA'])
        #model.fit(test,train, epochs = 100, batch_size=2)
        #model.fit(np.array(train[predictors]), np.array(train['Nxt_BA']), verbose=0, epochs=100)
    
        preds = model.predict(test[predictors]) 
        
        preds = pd.Series(preds, index=test.index)
        
        together = pd.concat([test['Nxt_BA'], preds], axis=1)
        together.columns = ['actual', 'prediction']
        
        bat_preds.append(together)
    return pd.concat(bat_preds)
                  

In [16]:
bat_ba_predictions = bat_ba_mrnn(batting, LSTM, bat_ba_preds)

ValueError: could not convert string to float: 'Alan\xa0Foster'

In [ ]:
#####
##### - TESTING THINGS OUT - WILL DELETE LATER
#####
    
        model.add(LSTM(units = 175, return_sequences = True, input_shape = (train.shape[1], 20)))
        model.add(Dropout(0.25))
        model.add(LSTM(units = 142, return_sequences = True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 125, return_sequences = True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 100, return_sequences = True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 75, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 75, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= False))
        model.add(Dense(units = 1))  

In [ ]:
#####
##### - TESTING THINGS OUT - WILL DELETE LATER
##### 
    
        model.add(LSTM(units = 175, return_sequences = True, input_shape = (None, 20)))
        model.add(Dropout(0.25))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= True, dropout=0.2, recurrent_dropout=0.15))
        model.add(LSTM(units = 50, return_sequences= False))
        model.add(Dense(units = 1)) 

In [ ]:
#####
# - NOTES
#####

In [ ]:
#####
# - NOTES
#####